# **Basic imports**

In [1]:
# initial imports 
import pandas as pd
from gekko import GEKKO
import numpy as np 
import seaborn as sns 
import logging
from functools import lru_cache
import ast 
import os 
import pandas as pd
from datetime import date
import logging
import matplotlib.pyplot as plt


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# **Coeff_ & intercepts**

In [2]:
# HCP DIGI 
hcp_digi_m1 = 6.01286785e+02
hcp_digi_m2 = -3.39198693e-01 
hcp_digi_m3 = 6.35363532e-05
hcp_digi_int_C = 67555.27887	

# HCP MAIL 
hcp_mail_m1 = 2.97917081e+03
hcp_mail_m2 = -1.71565008e+00 
hcp_mail_int_c = -48721.27889

# DTC DISPLAY 

dtc_disp_m1 = 3.45276779e+04
dtc_disp_m2 =  -1.65277192e+01  
dtc_disp_m3 = 5.93452413e-03
dtc_disp_m4 =  -9.42046720e-07
dtc_disp_intc = 192670.1652	

# # HCP  DISPLAY 
# hcp_disp_m1 =  4.11445774e-01 
# hcp_disp_m2 =  1.27940560e+02
# hcp_disp_m3 = -2.22402089e-01
# hcp_disp_m4 =  1.71248457e-04
# hcp_disp_m5 = -6.33547691e-08
# hcp_disp_m6 = 9.13303494e-12
# hcp_disp_intc = 817530.438

# -- field calls 

# model coeff_
f_m1 = 1.10785653e-03  
f_m2 = 3.42403093e+00 
f_m3 = -4.55404565e-04  
f_m4 = 2.57919190e-08 
f_m5 = -5.53661654e-13
f_Intercept_ = 239297
f_mROI_latch = 1.005172531

# hcp paid search ( corfficients & intercept )
hcp_m1 = 2.83578835e+04  
hcp_m2 = -1.67861082e+01 
hcp_int_c = -572151.0421

#DTC PAID SEARCH ( corfficients & intercept )
dtc_p_search_m1 = 1.71479676e+03 
dtc_p_search_m2 = -1.75357980e-01
dtc_p_search_int_c = -245074.9513

# Speaker program Person 

sp_p_m1 = 4.76886710e+01  
sp_p_m2 =  -3.09063497e-02 
sp_p_m3 =  6.78852771e-06
sp_p_intc = -744.7706042

# Speaker program Virtual 
sp_v_m1 = 4.21116616e+02 
sp_v_m2 = -2.80086720e-01 
sp_v_m3 = 5.98234666e-05
sp_v_intc = 45819.06696



# GNA CALLS 
gna_call_m1 = 1.53789502e+04
gna_call_m2 =  -1.56081127e+01  
gna_call_m3 = 7.02161425e-03 
gna_call_m4 = -1.17091134e-06
gna_call_intc = -996495.5305

# DTC SOCIAL 
dtc_social_m1 = 1.34750048e+04  
dtc_social_m2 = 9.07146577e+00 
dtc_social_m3 = -1.35022365e-02 
dtc_social_m4 =  5.48239513e-06 
dtc_social_m5 = -7.59241220e-10

dtc_social_intc = -1419139.752

# **data read**

In [63]:
data = pd.read_excel('./Excel_data_input/Reports.xlsx' , sheet_name='current_val')
pd.options.display.float_format = "{:.2f}".format
data.head(11)

,Channel Names,Current Spend,Saturation Point
0,field call,52891109.13,51251485
1,dtc paid search,6100000.38,6100000
2,hcp paid search,195203.29,238929
3,hcp_digital_spend,1468985.00,1468985
4,hcp_mail_spend,130745.01,192980
5,dtc_display_spend,1440000.05,2027520
6,hcp_display_spend,1036398.00,1242641
7,spk_person_spend,1008083.93,1008084
8,spk_virtual_spend,1260916.07,1260916
9,gna_call_spend,1100000.00,1100000


In [5]:
data['Current Spend'].sum()  # total current spend 

68167440.85520014

In [6]:
data['Saturation Point'].sum()  # sum of saturation points

67885268

In [64]:
# Total_budget = data['Current Spend'].sum()
# budget_10pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.1 )  # -10%
# budget_20pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.2 )
# budget_30pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.3 )
# budget_40pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.4 )
# budget_50pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.5 )
# budget_60pcnt = data['Current Spend'].sum() - ( data['Current Spend'].sum() * 0.6 )
# print('Total Budget  : ' , Total_budget)
# print('-10 % ' , budget_10pcnt)
# print('-20 % ' ,budget_20pcnt)
# print('-30 % ' ,budget_30pcnt )
# print('-40 % ' ,budget_40pcnt)
# print('-50 % ' ,budget_50pcnt)
# # print('-10 % ' ,budget_60pcnt)


# **Function 1**

In [12]:

# pcnt = 0.5
# total_budget = 34083720



def gekko_ (total_budget , pcnt ) :
    '''
        Function name : gekko_ ( function 1 : for Budget allocation)
        Inputs :         
            Total budget : int
            
        output : returns model  : Gekko model 
                        Constant vars : dict  
    
    '''
    
    model_2 = GEKKO()
    total_spend = model_2.Const(total_budget)
    # Constant spend of Websites 
    constant_values = {'hcp_website_spend_' : 483486 ,'dtc_website_spend_' : 400000 , 'hcp_website_revenue_' : 6800000 ,'dtc_website_revenue_' :  9600000 } 

    ##  revenue of websites
    fild_spend = model_2.Var(value=51251485+(-pcnt*51251485) , lb=51251485+(-pcnt*51251485)  , ub = 51251485+(pcnt*51251485), name='Field call spend' )   
    dp_spend = model_2.Var(value= 6100000+(-pcnt*6100000) , lb= 6100000+(-pcnt*6100000), ub = 6100000+(pcnt*6100000) , name='dtc paid search')     # change in upper boundary
    hcp_spend = model_2.Var(value=  238929+(-pcnt*238929), lb= 238929+(-pcnt*238929) , ub= 238929+(pcnt*238929), name='hcp paid search' )   # hcp paid search  
    hcp_digital_spend = model_2.Var(value=  1468985+(-pcnt*1468985), lb= 1468985+(-pcnt*1468985)  , ub=1468985+(pcnt*1468985) , name='hcp digital spend' ) 
    hcp_mail_spend = model_2.Var(value=  192980+(-pcnt*192980)  , lb= 192980+(-pcnt*192980)  , ub=  192980+(pcnt*192980) , name='hcp mail spend' )
    dtc_display_spend = model_2.Var(value=  2027520+(-pcnt*2027520), lb= 2027520+(-pcnt*2027520)   , ub= 2027520+(pcnt*2027520)  , name='dtc display spend'  ) 
    hcp_display_spend = model_2.Var(value=  1242641+(-pcnt*1242641)  , lb=  1242641+(-pcnt*1242641)   , ub=1242641+(pcnt*1242641), name='hcp display spend' ) 
    spk_person_spend = model_2.Var(value=   1008084+(-pcnt*1008084) , lb= 1008084+(-pcnt*1008084)   , ub= 1008084+(pcnt*1008084) , name='spk person spend') 
    spk_virtual_spend = model_2.Var(value=  1260916+(-pcnt*1260916), lb=1260916+(-pcnt*1260916), ub= 1260916+(pcnt*1260916), name='spk virtual spend')     # changes in lower boundary 
    gna_Calls_spend = model_2.Var(value=  1100000+(-pcnt*1100000), lb= 1100000+(-pcnt*1100000) , ub= 1100000+(pcnt*1100000), name='gna call spend')  
    dtc_social_spend = model_2.Var(value= 1993728+(-pcnt*1993728), lb= 1993728+(-pcnt*1993728) , ub=  1993728+(pcnt*1993728) , name='dtc_social_spend')  


    # sqrts  : Sqrt of channel as we have use SQRT + poly transformation 
    Field_sqrt = fild_spend**0.5       
    dp_sqrt = dp_spend**0.5
    hcp_sqrt = hcp_spend**0.5   # hcp paid search 
    hcp_digital_sqrt = hcp_digital_spend**0.5
    hcp_mail_Sqrt = hcp_mail_spend**0.5 
    dtc_display_sqrt = dtc_display_spend**0.5
    spk_person_sqrt = spk_person_spend**0.5 
    spk_virtual_Sqrt = spk_virtual_spend**0.5
    gna_call_sqrt = gna_Calls_spend**0.5 
    dtc_social_sqrt  = dtc_social_spend**0.5 

    # Equations 

    y_fild_calls = (Field_sqrt*f_m1) + (Field_sqrt**2)*(f_m2) + (Field_sqrt**3)*(f_m3) + (Field_sqrt**4)*(f_m4) + (Field_sqrt**5)*(f_m5) + f_Intercept_ # Revenue Field calls

    # DTC PAID SEARCH 
    y_dp_search = dp_sqrt*(dtc_p_search_m1) + ((dp_sqrt**2)*(dtc_p_search_m2))  + dtc_p_search_int_c    # Revenue  DTC paid search

    y_hcp_search = hcp_sqrt*(hcp_m1) + ( (hcp_sqrt**2)* hcp_m2) + hcp_int_c # Revenue HCP search 

    # hcp digital 
    y_hcp_digital  = hcp_digital_sqrt*(hcp_digi_m1) +  (hcp_digital_sqrt**2)*hcp_digi_m2 + (hcp_digital_sqrt**3)*hcp_digi_m3 + hcp_digi_int_C

    # HCP MAIL
    y_hcp_mail = hcp_mail_Sqrt*hcp_mail_m1 + (hcp_mail_Sqrt**2)*hcp_mail_m2 + hcp_mail_int_c

    # DTC DISPLAY 
    y_dtc_display = dtc_display_sqrt*dtc_disp_m1 + (dtc_display_sqrt**2)*dtc_disp_m2 + (dtc_display_sqrt**3)*dtc_disp_m3 + (dtc_display_sqrt**4)*dtc_disp_m4 + dtc_disp_intc

    # HCP DISPLAY 
    y_hcp_display = model_2.log(hcp_display_spend)*5628278.88 -54739893.15968231
    
    # speaker program-person
    y_spk_person = spk_person_sqrt*sp_p_m1 + (spk_person_sqrt**2)*(sp_p_m2) + (spk_person_sqrt**3)*(sp_p_m3) + sp_p_intc

    # Speaker program Virtual 
    y_spk_virtual = spk_virtual_Sqrt*sp_v_m1 + (spk_virtual_Sqrt**2)*(sp_v_m2) + (spk_virtual_Sqrt**3)*(sp_v_m3) + sp_v_intc

    # GNA CALLS 
    y_gna_Calls = gna_call_sqrt*gna_call_m1 + (gna_call_sqrt**2)*(gna_call_m2)+ (gna_call_sqrt**3)*(gna_call_m3)+ (gna_call_sqrt**4)*(gna_call_m4) + gna_call_intc

    # DTC SOCIAL 
    y_dtc_social = (dtc_social_sqrt * dtc_social_m1) + (dtc_social_sqrt**2)*(dtc_social_m2) + (dtc_social_sqrt**3)*(dtc_social_m3) + (dtc_social_sqrt**4)*(dtc_social_m4) + (dtc_social_sqrt**5)*(dtc_social_m5) + dtc_social_intc



    model_2.Equations([fild_spend + dp_spend + hcp_spend + hcp_digital_spend + hcp_mail_spend + dtc_display_spend + hcp_display_spend + spk_person_spend + spk_virtual_spend + gna_Calls_spend +dtc_social_spend <= total_spend] )  # sum of spend on all channels == Total Spend

    model_2.Maximize(y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual + y_gna_Calls + y_dtc_social  )    # #total revenue should be max 

    model_2.solve(disp=False) 
    
    return model_2 , constant_values

## prediction  (main function): 

In [37]:
# print( 'PRedicted Revenue :  ' , y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual+ y_gna_Calls+ y_dtc_social)
# print('Optimizer Maximized Result : ' , model_2.options.OBJFCNVAL)

In [65]:
def Output_revenue(total_budget , pcnt , Title , path='./'):
       
        model_2 , constant_vars = gekko_(total_budget=total_budget , pcnt= pcnt)
        print(constant_vars)
        
        l = [ x[0] for x in model_2._variables]
        Field_sqrt = l[0]**0.5
        dp_sqrt = l[1]**0.5  
        hcp_sqrt =l[2]**0.5 
        hcp_digital_sqrt =  l[3]**0.5
        hcp_mail_Sqrt = l[4]**0.5
        dtc_display_sqrt = l[5]**0.5 
        hcp_display_allocated_Spend = l[6]       # changed model ( for hcp display )
        spk_person_sqrt = l[7]**0.5
        spk_virtual_Sqrt =  l[8]**0.5
        gna_call_sqrt =l[9]**0.5 
        dtc_social_sqrt  = l[10]**0.5 
        
        global y_fild_calls 
        global y_dp_search 
        global y_hcp_search 
        global y_hcp_digital 
        global y_hcp_mail   
        global y_dtc_display 
        global y_hcp_display 
        global y_spk_person 
        global y_spk_virtual 
        global y_gna_Calls 
        global y_dtc_social       
        
        y_fild_calls = (Field_sqrt*f_m1) + (Field_sqrt**2)*(f_m2) + (Field_sqrt**3)*(f_m3) + (Field_sqrt**4)*(f_m4) + (Field_sqrt**5)*(f_m5) + f_Intercept_ # Revenue Field calls

        # DTC PAID SEARCH 
        y_dp_search = dp_sqrt*(dtc_p_search_m1) + ((dp_sqrt**2)*(dtc_p_search_m2))  + dtc_p_search_int_c    # Revenue  DTC paid search

        y_hcp_search = hcp_sqrt*(hcp_m1) + ( (hcp_sqrt**2)* hcp_m2) + hcp_int_c # Revenue HCP search 

        # hcp digital 
        y_hcp_digital  = hcp_digital_sqrt*(hcp_digi_m1) +  (hcp_digital_sqrt**2)*hcp_digi_m2 + (hcp_digital_sqrt**3)*hcp_digi_m3 + hcp_digi_int_C

        # HCP MAIL
        y_hcp_mail = hcp_mail_Sqrt*hcp_mail_m1 + (hcp_mail_Sqrt**2)*hcp_mail_m2 + hcp_mail_int_c

        # DTC DISPLAY 
        y_dtc_display = dtc_display_sqrt*dtc_disp_m1 + (dtc_display_sqrt**2)*dtc_disp_m2 + (dtc_display_sqrt**3)*dtc_disp_m3 + (dtc_display_sqrt**4)*dtc_disp_m4 + dtc_disp_intc

        # HCP DISPLAY 
        y_hcp_display = np.log(hcp_display_allocated_Spend)*5628278.88 -54739893.15968231


        # # speaker program-person
        y_spk_person = spk_person_sqrt*sp_p_m1 + (spk_person_sqrt**2)*(sp_p_m2) + (spk_person_sqrt**3)*(sp_p_m3) + sp_p_intc

        # Speaker program Virtual 
        y_spk_virtual = spk_virtual_Sqrt*sp_v_m1 + (spk_virtual_Sqrt**2)*(sp_v_m2) + (spk_virtual_Sqrt**3)*(sp_v_m3)+ sp_v_intc

        # # GNA CALLS 
        y_gna_Calls = gna_call_sqrt*gna_call_m1 + (gna_call_sqrt**2)*(gna_call_m2)+ (gna_call_sqrt**3)*(gna_call_m3)+ (gna_call_sqrt**4)*(gna_call_m4) + gna_call_intc

        # DTC SOCIAL 
        y_dtc_social = (dtc_social_sqrt * dtc_social_m1) + (dtc_social_sqrt**2)*(dtc_social_m2) + (dtc_social_sqrt**3)*(dtc_social_m3) + (dtc_social_sqrt**4)*(dtc_social_m4) + (dtc_social_sqrt**5)*(dtc_social_m5) + dtc_social_intc

        return generate_excel(Title=Title , path=path , constant_vars = constant_vars) 


In [66]:
result = Output_revenue(total_budget=68167440.85520014 , pcnt=0.0 ,Title = 'abcd')

{'hcp_website_spend_': 483486, 'dtc_website_spend_': 400000, 'hcp_website_revenue_': 6800000, 'dtc_website_revenue_': 9600000}


In [67]:
result.head(15)

,Channels,mROI at Current Spend,Current Spend,Allocated Budget,% change in Spend,Current Revenue,Predicted Revenue,% change in revenue
0,HCP Paid Search,13.98,195203.29,238929.00,22.40,8804973.37,9278597.63,5.38
1,Dtc Display,6.06,1440000.05,2027520.00,40.80,26100258.36,29107105.82,11.52
2,HCP Display,5.85,1036398.00,1242641.00,19.90,23737244.82,24240334.54,2.12
3,DTC Social,2.63,1536000.06,1993728.00,29.80,14194307.87,15213869.23,7.18
4,HCP Email,2.32,130745.01,192980.00,47.60,809427.92,928927.08,14.76
5,Field calls,0.65,52891109.13,51251485.00,-3.10,66837648.68,65970013.59,-1.30
6,GNA Calls,0.25,1100000.00,1100000.00,0.00,4627234.10,4648121.09,0.45
7,DTC Paid Search,0.16,6100000.38,6100000.00,-0.00,2960204.03,2920476.94,-1.34
8,HCP Digital,0.02,1468985.00,1468985.00,0.00,406099.50,411169.57,1.25
9,Speaker Program Virtual,0.01,1260916.07,1260916.00,-0.00,245515.21,250230.66,1.92


In [59]:
# constant_values = {'hcp_website_spend_' : 483486 ,'dtc_website_spend_' : 400000 , 'hcp_website_revenue_' : 6800000 ,'dtc_website_revenue_' :  9600000 } 

In [17]:
# channels = []
# Spend = []
# revenue = [] 
# constant_values = {'hcp_website_spend_' : 483486 ,'dtc_website_spend_' : 400000 , 'hcp_website_revenue_' : 6800000 ,'dtc_website_revenue_' :  9600000 } 

# for i in constant_values.keys():
#     if '_spend_' in i :
#         channels.append(i.split('_spend_')[0])
#         Spend.append(constant_values[i])
        
#     else:
#         revenue.append(constant_values[i])
    
# print(channels)
# print(Spend)
# print(revenue)
            

# **Excel report Generation**

In [60]:
from datetime import date

def generate_excel(Title , path , constant_vars):
    '''
        AIM : This function will help to generate Excel report
        Inputs : 
            Title : Unique name of file 
            Path : path to store file ..
    
    '''
    
    today = date.today()
    result_dict = {    
                'Channels' :   ['Field calls' , 'DTC Paid Search' , 'HCP Paid Search' , 'HCP Digital' , 'HCP Email' , 'Dtc Display' , 'HCP Display' ,'Speaker Program Person' , 
                                'Speaker Program Virtual' , 'GNA Calls' , 'DTC Social' ] ,         
                'mROI at Current Spend' : [] ,         
                'Current Spend' : [] ,  
                'Allocated Budget' : [x[0] for x in model_2._variables] , 

                'Predicted Revenue' : [y_fild_calls ,y_dp_search , y_hcp_search ,y_hcp_digital , y_hcp_mail ,  y_dtc_display ,y_hcp_display , y_spk_person ,  y_spk_virtual ,y_gna_Calls ,  y_dtc_social ] , 
                'Current Revenue' : [] 
                
            }        

    for i in result_dict['Channels'] : 
        df = pd.read_excel('./data/data_new.xlsx' , sheet_name=i, 
                        dtype={'% Change in Spend':str} )
        # print(np.where(df['% Change in Spend'] == '7.528699885739343e-16'))
        Current_data_ = df[df['% Change in Spend'] == '7.528699885739343e-16']
        result_dict['Current Spend'].append(Current_data_['Total Spend'].values[0])
        result_dict['Current Revenue'].append(Current_data_['Revenue'].values[0])
        result_dict['mROI at Current Spend'].append(Current_data_.iloc[:,-1].values[0]) 
        
    ## Add constant Values 
    result_dict['Channels'].append('HCP WEBSITE')
    result_dict['Current Revenue'].append(constant_vars['hcp_website_revenue_'])
    result_dict['Current Spend'].append(constant_vars['hcp_website_spend_'])
    result_dict['Allocated Budget'].append(constant_vars['hcp_website_spend_'])
    result_dict['Predicted Revenue'].append(constant_vars['hcp_website_revenue_'])
    result_dict['mROI at Current Spend'].append(0)
    
    
    
    result_dict['Channels'].append('DTC WEBSITE')    
    result_dict['Current Spend'].append(constant_vars['dtc_website_spend_'])    
    result_dict['Current Revenue'].append(constant_vars['dtc_website_revenue_'])
    result_dict['Allocated Budget'].append(constant_vars['dtc_website_spend_'])    
    result_dict['Predicted Revenue'].append(constant_vars['dtc_website_revenue_'])
    result_dict['mROI at Current Spend'].append(0)
    
    result_df = pd.DataFrame(result_dict)
    pd.options.display.float_format = "{:.2f}".format
    result_df = result_df.sort_values(by=['mROI at Current Spend'] , ascending=False)
    result_df['% change in revenue'] = ((result_df['Predicted Revenue'] - result_df['Current Revenue'] ) / result_df['Current Revenue'] )* 100
    result_df['% change in Spend'] = ((result_df['Allocated Budget'] - result_df['Current Spend'] ) / result_df['Current Spend'] )* 100

    additional_row = { 'Channels' : 'Total' , 'mROI at Current Spend' : np.nan , 'Current Spend' : result_df['Current Spend'].sum() , 
                    'Allocated Budget' : result_df['Allocated Budget'].sum(), 'Predicted Revenue' :result_df['Predicted Revenue' ].sum() , 'Current Revenue' :result_df['Current Revenue'].sum() ,
                     '% change in revenue' : np.nan, '% change in Spend' : np.nan }

    # arrnaging cols   
    result_df =result_df.append(additional_row , ignore_index=True)
    cols_sequence = ['Channels', 'mROI at Current Spend', 'Current Spend',
        'Allocated Budget', '% change in Spend', 'Current Revenue','Predicted Revenue', 
        '% change in revenue']
    result_df = result_df[cols_sequence]

    result_df.to_excel(f'{path}{Title}_Optimizer_Reports{today.strftime("%d_%m_%Y")}.xlsx')

    return result_df


In [69]:
# driver code 
# b = generate_excel('__50' , path='./fuct1 reports/')
# b.head()

## **results __ optimizer**

In [71]:
# print( 'PRedicted Revenue :  ' , y_fild_calls + y_dp_search + y_hcp_search + y_hcp_digital + y_hcp_mail + y_dtc_display + y_hcp_display + y_spk_person + y_spk_virtual+ y_gna_Calls+ y_dtc_social)
# print('Optimizer Maximized Result : ' , model_2.options.OBJFCNVAL)

In [72]:
# print(' y_fild_calls : ' , y_fild_calls)
# print(' y_dp_search : ' , y_dp_search)
# print(' y_hcp_search : ' , y_hcp_search)
# print(' y_hcp_digital : ' , y_hcp_digital)
# print(' y_hcp_mail : ' , y_hcp_mail)
# print(' y_dtc_display : ' , y_dtc_display)
# print(' y_hcp_display : ' , y_hcp_display)
# print('y_speaker_person' , y_spk_person)
# print(' y_spk_virtual : ' , y_spk_virtual)
# print(' y_gna_Calls : ' , y_gna_Calls)
# print(' y_dtc_social : ' , y_dtc_social)

In [21]:
# optimizer(js4)

In [ ]:
if 'hcp dipslay' in channel_name:
    == 'HCP DISPLAY'

# **Cache**

## **Cache with primary memory**

    - data will be lost after restarting the system

In [73]:
# from functools import lru_cache

# @lru_cache(maxsize=10)
def optimizer(data) : 

    df = pd.read_json(data)
    channels_list = df.channel_name.values 
    Variables_dict = {} 
    equations_dict = {} 
    TOTAL_BUDGET = df['total_budget'].unique()[0]

    model=GEKKO() 
    
    for _ , row_ in df.iterrows():
        Variables_dict[row_['channel_name']] =  model.Var(value= row_['current_spend'], lb=row_['min_spend'],ub=row_['max_spend'] , name= row_['channel_name'] )
        
        def build_eq(row_ , knob=None) :
            equations_dict[row_['channel_name']] = model.Equation( Variables_dict[row_['channel_name']]  < row_['max_spend'] )  

        build_eq(row_ = row_ )  # constrains 
        model.Maximize( ( model._variables[_] * row_['term_1 (a)'] + row_['term_2 (b)']  ) ) ## Objective Function n

    model.Equation(sum(model._variables) == TOTAL_BUDGET) # constrains    
    model.solve(disp=False) 
  
    result_dictionary = dict(zip(channels_list , model._variables))
    result_dictionary['Objective'] = model.options.objfcnval
    print(result_dictionary)
    return result_dictionary


In [76]:
## Driver code for above cache function 


# demo data 
json_data = '''{"channel_name":{"0":"Dtc_display","1":"DTC Social","2":"Dtc_paid_service"},
"function_type":{"0":"log-log","1":"log-level","2":"log-level"},
"r-square":{"0":0.9738176647,"1":0.97559292,"2":0.9755297402},
"term_1 (a)":{"0":5.555555555,"1":0.33439696,"2":0.39308951},
"term_2 (b)":{"0":4.90066613,"1":12.30889356,"2":8.64657667},
"term_3 (intercept)":{"0":null,"1":null,"2":null},
"current_spend":{"0":100000,"1":100000,"2":100000},
"min_spend":{"0":100000,"1":100000,"2":100000},
"max_spend":{"0":300000,"1":480000,"2":220000},
"total_budget":{"0":500000,"1":500000,"2":500000},
"threshould":{"0":0.2,"1":0.4,"2":0.4},
"optimized_spend":{"0":null,"1":null,"2":null},
"optimized_revenue":{"0":null,"1":null,"2":null}}'''

optimizer(json_data)   # without using cache

{'Dtc_display': [300000.0],
 'DTC Social': [100000.0],
 'Dtc_paid_service': [100000.00095],
 'Objective': -1739441.17}

## **Cache with secondary memory**

    - create new log file to store logs & always search data into the file if it is available already*

In [10]:
import pandas as pd
from gekko import GEKKO
import numpy as np 
import seaborn as sns 
import logging
from functools import lru_cache
import ast 
import os 

import matplotlib.pyplot as plt

In [18]:
pwd

'd:\\MMX\\ALL Channel models 21-03-2022\\12-04-22'

In [16]:
demo_data= pd.read_excel('demo_data/Copy of ResponseCurves_for_Simulator.xlsx' , sheet_name='DEMO')
demo_data.head()

,channel_name,function_type,r-square,term_1 (a),term_2 (b),current_spend,min_spend,max_spend,total_budget,threshould
0,Dtc_display,log-log,0.973818,4.588158,4.900666,100000,100000,300000,500000,0.2
1,DTC Social,log-level,0.975593,0.334397,12.308894,100000,100000,480000,500000,0.4
2,Dtc_paid_service,log-level,0.975530,0.393090,8.646577,100000,100000,220000,500000,0.4


In [17]:

logging.basicConfig( format='%(asctime)s:%(levelname)s:%(message)s' ,
 datefmt= '%d / %m / %Y  %I:%M:%S:%p  ' , 
                         filename='./logsss.log' ,level=logging.INFO)

In [23]:
# Optimizer function 

logFilePAth = 'logs_new.log'    # log file path 


def match_data(catch_data_str:str , input_data : dict ):
    '''
        catheddata : row string from log file : type :string 
        input_data : user input data in form of dict 
        AIM : Check if input data matched with logFile data if yes then return result , if no then return None 
                Fuction called inside the Optimizer function.. 
    '''

    d_ = catch_data_str.split('INFO:')[1]
    catch_data_dict = ast.literal_eval(d_)
    if catch_data_dict['data_dict'] == input_data:
        return catch_data_dict['result']
    else:
        pass 

    
def optimizer(data): 
    '''
        data : accept data in Json format 
        returns : optimized channel spends , type :  dict 
        function called inside : match_data()   
        
    '''
    
    df = pd.read_json(data)
    data_dict = df.to_dict()

    ## catch operation 
    # read the log file 
    with open(logFilePAth, mode='a+') as f_ : 
        file_data = f_.readlines()
        f_.close() 
        
    # if cache is not empty 
    if file_data:
        # iterate over the data present in file row by row 
        for row in file_data :    
            result = match_data(catch_data_str=row , input_data=data_dict)  
            # calling function 'macth_data' , to check if data is present or not , if yes , store result in 'result' variable 
            if result : 
                return result    # if data is present (return result ) else : run model 
                          
    ## if data isn't present in catch                     
    channels_list = df.channel_name.values 
    Variables_dict = {}     # store gekko variables with channel name as key 
    equations_dict = {}     # store model Equations with channel name as key 
    TOTAL_BUDGET = df['total_budget'].unique()[0]   # Ovarall Total Buget 

    model=GEKKO() 

    for _ , row_ in df.iterrows():
        # iterate over the each record in dataframe & create model variable based on channel constrains (min & max spends)
        Variables_dict[row_['channel_name']] =  model.Var(value= row_['current_spend'], lb=row_['min_spend'],ub=row_['max_spend'] , name= row_['channel_name'] )
        
        def build_eq(row_) :
            # add constrains per channel ( channel_x spend <  channel_X maxspend)
            equations_dict[row_['channel_name']] = model.Equation( Variables_dict[row_['channel_name']]  < row_['max_spend'] )  

        build_eq(row_ = row_ )  # constrains 
        model.Maximize( ( model._variables[_] * row_['term_1 (a)'] + row_['term_2 (b)']  ) ) # Objective Function

    model.Equation(sum(model._variables) == TOTAL_BUDGET) # constrains : sum of channel spend should be equal to Total Budget  
    model.solve(disp=False) 

    result_dictionary = dict(zip(channels_list , model._variables))
    result_dictionary['Objective'] = model.options.objfcnval

    logging.info({'data_dict':data_dict , 'result' : result_dictionary })     # logging data + result in logg file ( for cache purpose )

    return result_dictionary

In [24]:
# driver code 
optimizer(demo_data.to_json()) 


{'Dtc_display': [300000.0],
 'DTC Social': [100000.0],
 'Dtc_paid_service': [100000.00094],
 'Objective': -1449221.8285}

In [25]:
optimizer(demo_data.to_json()) 

{'Dtc_display': [300000.0],
 'DTC Social': [100000.0],
 'Dtc_paid_service': [100000.00094],
 'Objective': -1449221.8285}

In [26]:
optimizer(demo_data.to_json()) 

{'Dtc_display': [300000.0],
 'DTC Social': [100000.0],
 'Dtc_paid_service': [100000.00094],
 'Objective': -1449221.8285}